In [1]:
from osgeo import gdal
import numpy as np
path = r"C:\Users\Tanay Jagannathan\Desktop\Pathing AI Data\odiac2022_1km_excl_intl_2101.tif\odiac2022_1km_excl_intl_2101.tif"
dataset = gdal.Open(path)

width = dataset.RasterXSize
height = dataset.RasterYSize
num_bands = dataset.RasterCount



print(width)
print(height)
print(num_bands)

43200
21600
1


In [43]:
from osgeo import gdal
import numpy as np
import gymnasium
from gymnasium import spaces

class CustomLearningEnvironment(gymnasium.Env):
    def __init__(self, dataset_path):
        super(CustomLearningEnvironment, self).__init__()

        self.dataset = gdal.Open(dataset_path)
        self.width = self.dataset.RasterXSize
        self.height = self.dataset.RasterYSize
        self.num_bands = self.dataset.RasterCount
        self.fuel = 15

        self.band_data = self.dataset.GetRasterBand(1).ReadAsArray()

        self.agent_pos=(2,2)
        self.starting_pos = self.agent_pos
        self.agent_path = []
        
        max_val = np.amax(self.band_data)
        self.action_space = spaces.Discrete(4)
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(self.height*self.width+2,))
        self.total_reward = 0
        
    def reset(self, seed = None):
        if seed is not None:
            np.random.seed(seed)
        self.agent_pos = (2,2)
        self.fuel = 15
        self.band_data = self.dataset.GetRasterBand(1).ReadAsArray()
        print(self.agent_path)
        self.agent_path= []
        self.total_reward = 0
        return self.getObservation(), {}

    def step(self, action):
        new_pos = self.getNextPosition(action)
        if self.isWithinBounds(new_pos):
            initial_value = self.band_data[self.agent_pos[0], self.agent_pos[1]] * 0.995
            next_value = self.band_data[new_pos[0], new_pos[1]] * 0.995

            reduction_amount = (self.band_data[self.agent_pos[0], self.agent_pos[1]] - initial_value) + (self.band_data[new_pos[0], new_pos[1]] - next_value)

            self.band_data[self.agent_pos[0], self.agent_pos[1]] = initial_value
            self.band_data[new_pos[0], new_pos[1]] = next_value

            self.agent_pos = new_pos
        else:
            reduction_amount = -50000
        
        
        
        self.agent_path.append(self.agent_pos)

        self.fuel= self.fuel - 1

        reward = self.calculateReward() + reduction_amount
        self.total_reward += reward
        done = self.isDone()
        return self.getObservation(), reward, done, {}, {}

    def isWithinBounds(self, pos):
        return (0 <= pos[0] < self.width) and (0 <= pos[1] < self.height)

    def getNextPosition(self, action):
        match action:
            case 0:
                return (self.agent_pos[0], self.agent_pos[1]+1)
            case 1:
                return (self.agent_pos[0], self.agent_pos[1]-1)
            case 2: 
                return (self.agent_pos[0]-1, self.agent_pos[1])
            case 3: 
                return (self.agent_pos[0]+1, self.agent_pos[1])
            case _:
                return

    def calculateReward(self):
        reward = 0
    # Give a large penalty if the agent is not at its starting position when fuel runs out
        if self.fuel == 0 and self.agent_pos != self.starting_pos:
            reward -= 1000
        return reward

    def isDone(self):
        return self.fuel == 0

    def getObservation(self):
        obs = np.concatenate((self.band_data.flatten() / np.max(self.band_data), np.array(self.agent_pos) / (self.width, self.height)))
        return obs

    def render(self, mode='human'):
        pass

    def close(self):
        pass

In [44]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import os


env_paths = []
global_path = r"C:\Users\Tanay Jagannathan\Desktop\Pathing AI Data\odiac2022_1km_excl_intl_2101.tif\odiac2022_1km_excl_intl_2101.tif"
save_path = r"C:\Users\Tanay Jagannathan\Desktop\Pathing AI Data\Test_Envs"

for name in os.listdir(save_path):
    env_paths.append(os.path.join(save_path, name))




viz_path = r'C:\Users\Tanay Jagannathan\Desktop\Pathing AI Data\Training 2\Visualizations'
envs = [DummyVecEnv([lambda: CustomLearningEnvironment(path)]) for path in env_paths]

model = PPO("MlpPolicy", envs[0], learning_rate = 0.01, verbose=2)

for i, env in enumerate(envs):
    model.set_env(env)
    model.learn(total_timesteps=100000)
    

    
    
    environ = env.envs[0]
    path = environ.agent_path
    total_reward = environ.total_reward
    print(path)
    print('done')

   # Generate a 5x5 array from the environment's data
    array = environ.band_data

   # Visualize the agent's path on the array
    visualize_path(array, path, os.path.join(viz_path, f'env{i}.png'), total_reward)
model.save('PPO_pathing')
print('done')




Using cpu device
[]
[(1, 2), (1, 1), (1, 0), (2, 0), (2, 0), (1, 0), (2, 0), (2, 1), (2, 0), (2, 1), (2, 2), (3, 2), (4, 2), (3, 2), (4, 2)]
[(1, 2), (2, 2), (2, 1), (2, 2), (3, 2), (3, 1), (2, 1), (1, 1), (1, 2), (0, 2), (1, 2), (1, 1), (0, 1), (0, 1), (0, 1)]
[(3, 2), (3, 3), (2, 3), (1, 3), (0, 3), (0, 3), (0, 4), (1, 4), (0, 4), (0, 4), (0, 4), (0, 4), (1, 4), (1, 3), (1, 2)]
[(2, 3), (2, 4), (2, 3), (2, 4), (1, 4), (1, 3), (1, 4), (2, 4), (2, 3), (2, 4), (1, 4), (2, 4), (3, 4), (2, 4), (2, 4)]
[(1, 2), (1, 1), (2, 1), (2, 2), (1, 2), (0, 2), (0, 2), (0, 1), (1, 1), (0, 1), (0, 2), (1, 2), (0, 2), (0, 2), (0, 1)]
[(3, 2), (3, 1), (3, 2), (3, 3), (2, 3), (2, 4), (2, 3), (2, 4), (2, 4), (2, 3), (3, 3), (3, 2), (2, 2), (1, 2), (2, 2)]
[(1, 2), (0, 2), (0, 3), (1, 3), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (1, 1), (1, 2), (2, 2), (2, 3), (2, 4), (1, 4)]
[(1, 2), (1, 1), (2, 1), (3, 1), (2, 1), (1, 1), (1, 0), (2, 0), (2, 1), (2, 2), (2, 1), (3, 1), (3, 0), (4, 0), (3, 0)]
[(2, 3), (1,

[(2, 3), (2, 4), (3, 4), (4, 4), (4, 4), (4, 4), (4, 3), (4, 4), (4, 4), (4, 4), (4, 3), (4, 4), (3, 4), (2, 4), (1, 4)]
[(1, 2), (0, 2), (0, 2), (0, 2), (0, 2), (0, 1), (1, 1), (1, 2), (0, 2), (0, 3), (0, 4), (0, 4), (0, 3), (0, 2), (0, 1)]
[(3, 2), (3, 1), (3, 0), (3, 1), (2, 1), (1, 1), (0, 1), (1, 1), (1, 0), (2, 0), (2, 1), (3, 1), (4, 1), (3, 1), (3, 0)]
[(3, 2), (3, 3), (3, 4), (4, 4), (4, 4), (3, 4), (3, 4), (3, 3), (3, 4), (3, 3), (4, 3), (3, 3), (4, 3), (4, 4), (4, 4)]
[(3, 2), (4, 2), (4, 3), (4, 3), (4, 2), (3, 2), (3, 3), (4, 3), (4, 4), (4, 3), (4, 3), (3, 3), (3, 2), (3, 3), (2, 3)]
[(2, 1), (3, 1), (4, 1), (4, 0), (3, 0), (3, 1), (3, 2), (2, 2), (3, 2), (4, 2), (4, 2), (4, 2), (4, 1), (4, 0), (3, 0)]
[(3, 2), (3, 1), (3, 2), (4, 2), (4, 1), (3, 1), (4, 1), (4, 0), (3, 0), (3, 0), (3, 1), (3, 0), (3, 0), (2, 0), (3, 0)]
[(1, 2), (0, 2), (0, 3), (1, 3), (1, 2), (2, 2), (3, 2), (4, 2), (4, 1), (4, 2), (3, 2), (3, 1), (2, 1), (2, 2), (1, 2)]
[(2, 1), (2, 0), (2, 0), (2, 0),

[(2, 3), (2, 2), (2, 1), (2, 2), (1, 2), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (2, 2), (3, 2), (3, 3), (2, 3)]
[(2, 1), (1, 1), (1, 0), (1, 0), (2, 0), (2, 1), (3, 1), (3, 0), (4, 0), (4, 0), (4, 1), (3, 1), (3, 0), (4, 0), (3, 0)]
[(3, 2), (4, 2), (3, 2), (2, 2), (3, 2), (4, 2), (3, 2), (3, 3), (3, 2), (4, 2), (4, 1), (4, 0), (4, 0), (4, 0), (3, 0)]
[(1, 2), (2, 2), (3, 2), (2, 2), (1, 2), (1, 1), (1, 0), (0, 0), (0, 1), (0, 0), (0, 1), (0, 0), (0, 1), (1, 1), (1, 0)]
[(2, 1), (2, 0), (2, 1), (3, 1), (4, 1), (4, 1), (4, 1), (4, 0), (3, 0), (3, 0), (2, 0), (2, 0), (2, 1), (1, 1), (1, 2)]
[(1, 2), (0, 2), (1, 2), (0, 2), (0, 3), (0, 2), (0, 2), (1, 2), (0, 2), (1, 2), (1, 1), (2, 1), (3, 1), (3, 2), (2, 2)]
[(3, 2), (3, 1), (3, 2), (3, 3), (4, 3), (4, 2), (4, 1), (4, 0), (3, 0), (4, 0), (3, 0), (4, 0), (4, 0), (4, 1), (4, 0)]
[(3, 2), (3, 3), (3, 4), (3, 3), (2, 3), (1, 3), (2, 3), (1, 3), (1, 2), (1, 3), (0, 3), (0, 3), (0, 2), (0, 3), (0, 4)]
[(3, 2), (2, 2), (3, 2), (3, 3),

[(3, 2), (4, 2), (4, 1), (4, 0), (4, 0), (4, 0), (3, 0), (4, 0), (3, 0), (2, 0), (1, 0), (0, 0), (0, 0), (0, 0), (1, 0)]
[(3, 2), (3, 3), (3, 2), (4, 2), (4, 2), (4, 3), (3, 3), (3, 4), (4, 4), (4, 4), (4, 4), (3, 4), (4, 4), (4, 3), (4, 4)]
[(2, 1), (3, 1), (4, 1), (3, 1), (2, 1), (3, 1), (2, 1), (3, 1), (3, 0), (4, 0), (3, 0), (3, 1), (3, 2), (3, 1), (2, 1)]
[(2, 3), (2, 2), (1, 2), (1, 3), (2, 3), (2, 2), (2, 3), (1, 3), (1, 2), (2, 2), (1, 2), (0, 2), (1, 2), (1, 1), (2, 1)]
[(3, 2), (3, 3), (2, 3), (2, 4), (2, 4), (3, 4), (4, 4), (4, 3), (3, 3), (3, 4), (4, 4), (4, 3), (4, 4), (4, 3), (4, 2)]
[(2, 3), (1, 3), (1, 4), (1, 3), (2, 3), (2, 2), (2, 1), (2, 2), (2, 3), (2, 4), (3, 4), (2, 4), (3, 4), (3, 3), (3, 4)]
[(2, 1), (2, 2), (2, 3), (1, 3), (1, 4), (2, 4), (3, 4), (3, 3), (2, 3), (3, 3), (3, 2), (4, 2), (4, 3), (4, 2), (4, 1)]
[(3, 2), (3, 1), (3, 0), (3, 1), (4, 1), (4, 2), (4, 1), (4, 0), (4, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 2), (4, 2)]
[(2, 1), (2, 2), (2, 1), (3, 1),

[(2, 3), (2, 2), (3, 2), (3, 3), (2, 3), (3, 3), (2, 3), (2, 4), (2, 3), (2, 4), (3, 4), (3, 4), (3, 3), (2, 3), (1, 3)]
[(2, 1), (2, 2), (2, 1), (3, 1), (3, 0), (2, 0), (2, 0), (2, 0), (1, 0), (2, 0), (2, 1), (2, 0), (2, 1), (3, 1), (2, 1)]
[(2, 3), (1, 3), (0, 3), (1, 3), (2, 3), (1, 3), (0, 3), (0, 2), (0, 3), (1, 3), (0, 3), (1, 3), (0, 3), (0, 2), (0, 2)]
[(2, 3), (2, 2), (3, 2), (3, 3), (2, 3), (3, 3), (4, 3), (3, 3), (4, 3), (4, 2), (4, 2), (4, 3), (4, 3), (4, 3), (4, 2)]
[(3, 2), (3, 1), (2, 1), (2, 0), (2, 1), (3, 1), (2, 1), (3, 1), (3, 0), (4, 0), (3, 0), (2, 0), (2, 1), (3, 1), (2, 1)]
[(1, 2), (1, 1), (1, 2), (2, 2), (2, 3), (1, 3), (0, 3), (0, 2), (0, 2), (0, 1), (0, 0), (0, 0), (0, 0), (0, 0), (1, 0)]
[(2, 1), (3, 1), (3, 2), (4, 2), (4, 2), (4, 1), (4, 0), (4, 1), (3, 1), (4, 1), (4, 1), (4, 0), (4, 0), (4, 0), (3, 0)]
[(2, 3), (2, 2), (1, 2), (1, 3), (1, 4), (1, 3), (1, 2), (2, 2), (1, 2), (1, 1), (1, 2), (1, 3), (0, 3), (0, 4), (1, 4)]
[(3, 2), (3, 3), (3, 4), (3, 4),

...

[(2, 3), (1, 3), (2, 3), (2, 2), (1, 2), (2, 2), (1, 2), (1, 1), (1, 2), (1, 3), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4)]
[(1, 2), (0, 2), (1, 2), (1, 3), (1, 4), (1, 4), (1, 3), (1, 4), (2, 4), (3, 4), (2, 4), (2, 3), (3, 3), (4, 3), (4, 2)]
[(3, 2), (2, 2), (1, 2), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4), (4, 3), (4, 3), (4, 2), (4, 1), (4, 0), (4, 1), (4, 0)]
------------------------------------------
| time/                   |              |
|    fps                  | 675          |
|    iterations           | 19           |
|    time_elapsed         | 57           |
|    total_timesteps      | 38912        |
| train/                  |              |
|    approx_kl            | 6.191258e-07 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.39        |
|    explained_variance   | 0            |
|    learning_rate        | 0.0003       |
|    loss                 | 3.23e+09     |
|    n_updates            | 1180   

[(1, 2), (0, 2), (1, 2), (1, 1), (1, 0), (1, 0), (0, 0), (0, 0), (0, 1), (1, 1), (1, 0), (0, 0), (0, 1), (1, 1), (1, 2)]
[(1, 2), (0, 2), (0, 3), (0, 2), (1, 2), (1, 3), (1, 2), (1, 3), (2, 3), (1, 3), (1, 4), (0, 4), (0, 4), (0, 4), (0, 3)]
[(2, 1), (2, 0), (2, 1), (2, 2), (2, 1), (2, 2), (3, 2), (3, 3), (2, 3), (2, 4), (3, 4), (4, 4), (3, 4), (3, 3), (4, 3)]
[(2, 1), (2, 0), (2, 0), (2, 0), (2, 0), (1, 0), (1, 0), (2, 0), (2, 1), (2, 0), (2, 0), (2, 0), (3, 0), (3, 1), (4, 1)]
[(2, 3), (2, 4), (3, 4), (2, 4), (2, 4), (2, 3), (2, 2), (2, 3), (3, 3), (3, 2), (3, 3), (2, 3), (2, 2), (1, 2), (0, 2)]
[(3, 2), (3, 3), (3, 4), (3, 4), (3, 3), (4, 3), (3, 3), (2, 3), (2, 4), (2, 3), (1, 3), (1, 4), (2, 4), (2, 4), (3, 4)]
[(2, 3), (3, 3), (4, 3), (4, 3), (4, 3), (3, 3), (2, 3), (3, 3), (3, 4), (3, 3), (3, 2), (2, 2), (3, 2), (3, 3), (2, 3)]
[(2, 3), (3, 3), (3, 2), (2, 2), (2, 1), (1, 1), (1, 2), (1, 1), (1, 2), (1, 1), (1, 2), (2, 2), (2, 1), (3, 1), (4, 1)]
[(3, 2), (3, 3), (3, 2), (4, 2),

[(2, 3), (2, 2), (2, 1), (3, 1), (4, 1), (4, 1), (4, 1), (4, 0), (3, 0), (2, 0), (2, 1), (2, 2), (3, 2), (3, 3), (3, 2)]
[(1, 2), (1, 3), (2, 3), (2, 2), (3, 2), (2, 2), (2, 3), (2, 2), (2, 3), (2, 2), (1, 2), (0, 2), (0, 2), (0, 3), (1, 3)]
[(3, 2), (2, 2), (2, 1), (2, 0), (2, 0), (2, 0), (2, 1), (3, 1), (4, 1), (4, 1), (4, 0), (3, 0), (3, 0), (2, 0), (2, 0)]
[(1, 2), (1, 1), (1, 0), (1, 1), (0, 1), (0, 0), (1, 0), (1, 0), (2, 0), (1, 0), (0, 0), (0, 0), (0, 0), (0, 1), (0, 2)]
[(2, 3), (2, 4), (1, 4), (0, 4), (0, 4), (0, 4), (0, 3), (0, 4), (0, 4), (0, 4), (1, 4), (0, 4), (0, 3), (1, 3), (0, 3)]
[(2, 3), (1, 3), (1, 4), (1, 4), (1, 4), (0, 4), (0, 4), (0, 4), (1, 4), (1, 4), (1, 3), (0, 3), (0, 4), (0, 4), (1, 4)]
[(2, 3), (2, 4), (2, 3), (3, 3), (2, 3), (2, 2), (1, 2), (1, 1), (0, 1), (0, 2), (0, 2), (0, 3), (1, 3), (1, 2), (2, 2)]
[(1, 2), (0, 2), (1, 2), (1, 3), (1, 2), (1, 3), (2, 3), (2, 2), (2, 1), (1, 1), (1, 0), (1, 0), (1, 1), (1, 2), (0, 2)]
[(2, 1), (2, 2), (1, 2), (0, 2),

[(1, 2), (2, 2), (2, 1), (2, 2), (2, 3), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4), (4, 4), (3, 4), (2, 4), (1, 4), (1, 3)]
[(2, 1), (2, 0), (2, 1), (2, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 0), (4, 1), (3, 1), (2, 1), (2, 2), (2, 3), (1, 3)]
[(2, 1), (1, 1), (1, 2), (1, 3), (0, 3), (0, 3), (0, 3), (1, 3), (1, 2), (2, 2), (1, 2), (1, 1), (1, 0), (1, 0), (1, 0)]
[(3, 2), (2, 2), (2, 3), (2, 2), (2, 1), (2, 2), (2, 3), (2, 2), (1, 2), (1, 1), (2, 1), (1, 1), (0, 1), (1, 1), (0, 1)]
[(3, 2), (3, 3), (3, 2), (3, 1), (4, 1), (4, 2), (3, 2), (4, 2), (3, 2), (4, 2), (3, 2), (2, 2), (3, 2), (3, 1), (3, 0)]
[(2, 3), (2, 2), (2, 3), (2, 4), (2, 4), (2, 4), (2, 4), (1, 4), (1, 4), (0, 4), (0, 4), (0, 4), (1, 4), (2, 4), (2, 3)]
[(2, 1), (2, 2), (2, 1), (2, 2), (2, 3), (2, 4), (2, 4), (2, 4), (1, 4), (2, 4), (2, 3), (3, 3), (3, 4), (3, 3), (3, 4)]
[(1, 2), (2, 2), (3, 2), (4, 2), (4, 1), (4, 2), (4, 3), (4, 2), (3, 2), (3, 1), (2, 1), (2, 2), (2, 3), (2, 4), (2, 4)]
[(1, 2), (2, 2), (3, 2), (4, 2),

KeyboardInterrupt: 

In [4]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt

def generate_array(dataset_path, save_path):


    dataset = gdal.Open(dataset_path)

    # Get the raster band
    band = dataset.GetRasterBand(1)

    # Read the data as a 2D array
    data = band.ReadAsArray()

    # Scale the data from grams of carbon per day to grams of CO2 per year



    height, width = data.shape
    while True:
        x_index = np.random.randint(0, width-4)

        y_index = np.random.randint(0, height-4)
        array = data[x_index:x_index+5, y_index:y_index+5]
        array = array*365.25
        if np.sum(array) > 912500:
            break
    geotransform = dataset.GetGeoTransform()
    projection = dataset.GetProjection()

    # Create a new GeoTIFF file
    driver = gdal.GetDriverByName("GTiff")
    output_dataset = driver.Create(save_path, 5, 5, 1, gdal.GDT_Float32)

    # Write the array to the new GeoTIFF file
    output_dataset.GetRasterBand(1).WriteArray(array)

    # Set the geotransform and projection of the new GeoTIFF file
    output_dataset.SetGeoTransform(geotransform)
    output_dataset.SetProjection(projection)

    # Close the datasets
    dataset = None
    output_dataset = None

    return save_path

In [27]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt


def visualize_path(array, path, save_path, total_reward):
    path = np.array(path)

    array_normalized = array / np.max(array)

    cmap = plt.get_cmap('hot')
    fig, ax = plt.subplots()
    im = ax.imshow(array_normalized, cmap=cmap, interpolation='nearest')
    ax.plot(path[:, 1], path[:, 0], color='green', linewidth=5)

    path_str = ', '.join(f'({x}, {y})' for x, y in path)
    plt.text(0, -1, path_str, ha='left', va='center', color='blue', size=10, wrap = True)

    plt.text(0, 5, str(total_reward), ha='left', va='center', color='blue', size=10, wrap = True)
    fig.colorbar(im, ax=ax)
    plt.savefig(save_path)

    plt.close(fig)

In [42]:
import os

global_path = r'C:\Users\Tanay Jagannathan\Desktop\Pathing AI Data\odiac2022_1km_excl_intl_2101.tif\odiac2022_1km_excl_intl_2101.tif'
save_dir = r'C:\Users\Tanay Jagannathan\Desktop\Pathing AI Data\Test_Envs'
for i in range(500):
    #Generate a 5x5 array from the global GeoTIFF file and save it as a GeoTIFF file

    geotiff_file = generate_array(global_path, os.path.join(save_dir, f'array{i+500}.tif'))
    print(f'{i+500}')
    


500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749


In [6]:
from osgeo import gdal
import numpy as np
dataset_path = r'C:\Users\Tanay Jagannathan\Desktop\Pathing AI Data\odiac2022_1km_excl_intl_2101.tif\odiac2022_1km_excl_intl_2101.tif'
dataset = gdal.Open(dataset_path)
band = dataset.GetRasterBand(1)
data = band.ReadAsArray()

print(f'{data.shape}')
while True:
    
    x_index = np.random.randint(0, 43200-4)

    y_index = np.random.randint(0, 21600-4)
    array = data[x_index:x_index+5, y_index:y_index+5]
    array = array*365.25
    if np.sum(array) > 912500:
        break


print(f'{x_index}')
print(f'{y_index}')

print(np.matrix(array))


(21600, 43200)
5713
11054
[[63538.938 60655.4   58058.44  53221.973 45950.1  ]
 [69885.664 66440.07  67300.22  61856.29  54725.77 ]
 [75035.1   71101.36  70200.586 67168.24  59030.883]
 [75259.63  71329.62  66631.83  65919.52  58449.445]
 [73472.02  70891.62  66799.32  63281.805 60768.59 ]]


In [ ]:
from osgeo import gdal
import numpy as np

array = gdal.Open(r"C:\Users\Tanay Jagannathan\Desktop\Pathing AI Data\Test_Envs\array499.tif").GetRasterBand(1).ReadAsArray()
path = [(1, 2), (2, 2), (2, 1), (2, 2), (2, 1), (2, 0), (2, 0), (2, 0), (2, 1), (2, 2)]
save_path = r'C:\Users\Tanay Jagannathan\Desktop\Pathing AI Data\Visualizations\test.png'
visualize_path(array, path, save_path)